In [ ]:
import torch

# Load the processed dataset
dataset_path = "edge_dataset_with_labels.pt"
processed_dataset = torch.load(dataset_path)

In [7]:
# Load the dataset
dataset = torch.load('edge_dataset_with_labels.pt')  # Replace with your path

# Compute class weights to handle imbalance
label_counter = Counter()
for graph in dataset:
    label_counter.update(graph.y.tolist())

# Class frequencies
num_class_0 = label_counter[0]
num_class_1 = label_counter[1]
total_samples = num_class_0 + num_class_1

# Compute class weights
weight_0 = total_samples / (2 * num_class_0)
weight_1 = total_samples / (2 * num_class_1)
class_weights = torch.tensor([weight_0, weight_1], dtype=torch.float)
print(f"Class 0 Weight: {weight_0}, Class 1 Weight: {weight_1}")

# Split dataset into train/test (80/20 split)
train_size = int(0.8 * len(dataset))
train_dataset = dataset[:train_size]
test_dataset = dataset[train_size:]

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Training graphs: {len(train_dataset)}, Testing graphs: {len(test_dataset)}")


/var/tmp/pbs.64362.bright04/ipykernel_3406836/1288772817.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset = torch.load('edge_dataset_with_labels.pt')  # Replace w

Class 0 Weight: 0.5123171043755379, Class 1 Weight: 20.796978281397546
Training graphs: 17199, Testing graphs: 4300


In [21]:
import torch
import torch.nn.functional as F
from torch.nn import Linear, Sequential, ReLU
from torch_geometric.nn import NNConv

class EdgeFailureGNN(torch.nn.Module):
    def __init__(self, in_node_feats, in_edge_feats, hidden_dim, out_dim=2):
        super(EdgeFailureGNN, self).__init__()

        # Edge-aware MLP for NNConv to compute weights
        self.edge_nn1 = Sequential(
            Linear(in_edge_feats, hidden_dim),
            ReLU(),
            Linear(hidden_dim, in_node_feats * hidden_dim)  # 3 * hidden_dim
        )
        self.edge_nn2 = Sequential(
            Linear(in_edge_feats, hidden_dim),
            ReLU(),
            Linear(hidden_dim, hidden_dim * hidden_dim)  # hidden_dim * hidden_dim
        )

        # NNConv layers
        self.conv1 = NNConv(in_node_feats, hidden_dim, self.edge_nn1, aggr='mean')
        self.conv2 = NNConv(hidden_dim, hidden_dim, self.edge_nn2, aggr='mean')

        # Edge classification MLP
        self.edge_mlp = Sequential(
            Linear(hidden_dim * 2, hidden_dim),
            ReLU(),
            Linear(hidden_dim, out_dim)  # Output: 2 classes (failure or non-failure)
        )

    def forward(self, x, edge_index, edge_attr):
        # NNConv layers for node embeddings
        x = F.relu(self.conv1(x, edge_index, edge_attr))
        x = F.relu(self.conv2(x, edge_index, edge_attr))

        # Edge classification: combine node embeddings at edge ends
        row, col = edge_index
        edge_features = torch.cat([x[row], x[col]], dim=1)  # Concatenate source & target node embeddings

        # Pass through edge MLP
        return self.edge_mlp(edge_features)


In [22]:
# Training Loop
epochs = 20
for epoch in range(1, epochs + 1):
    loss = train(model, train_loader)
    print(f"Epoch {epoch}, Loss: {loss:.4f}")


RuntimeError: The size of tensor a (2358) must match the size of tensor b (786) at non-singleton dimension 0

In [17]:
import torch.nn.functional as F

def train(model, loader):
    model.train()
    total_loss = 0

    for data in loader:  # Iterate over each batched graph
        data = data.to(device)
        optimizer.zero_grad()
        
        # Forward pass
        out = model(data.x, data.edge_index, data.edge_attr)
        
        # Loss calculation: match outputs with edge labels
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    return total_loss / len(loader)

# Train the model for multiple epochs
epochs = 20
for epoch in range(1, epochs + 1):
    loss = train(model, train_loader)
    print(f"Epoch {epoch}, Loss: {loss:.4f}")


RuntimeError: shape '[-1, 3, 64]' is invalid for input of size 151168

In [ ]:
from torch.utils.data.sampler import WeightedRandomSampler

# Step 1: Recompute graph-level labels
# A graph is labeled as 1 (positive class) if it contains any failed edges (label == 1)
train_graph_labels = [1 if graph.y.sum() > 0 else 0 for graph in train_dataset] 

# Step 2: Compute class weights (inverse proportional to class counts)
class_counts = torch.bincount(torch.tensor(train_graph_labels))
class_weights = 1.0 / (class_counts + 1e-6)
print(f"Class Counts: {class_counts}")
print(f"Class Weights: {class_weights}")

# Step 3: Assign weights to graphs in the training dataset
train_graph_weights = [class_weights[label] for label in train_graph_labels]

# Step 4: Create a weighted random sampler for oversampling minority graphs
train_sampler = WeightedRandomSampler(train_graph_weights, num_samples=len(train_dataset), replacement=True)

# Step 5: Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Oversampling minority class in the training dataset completed.")


In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear, ModuleList, Dropout
from torch_geometric.nn import GCNConv, GATv2Conv

class EdgePredictionGNN(torch.nn.Module):
    def __init__(self, input_dim, edge_dim, hidden_dim, output_dim, num_layers=3, dropout=0.5):
        super(EdgePredictionGNN, self).__init__()
        self.num_layers = num_layers
        self.dropout = Dropout(dropout)
        
        # Node feature encoder
        self.node_encoder = Linear(input_dim, hidden_dim)
        
        # Edge feature encoder
        self.edge_encoder = Linear(edge_dim, hidden_dim)
        
        # GNN layers (GATv2Conv supports edge attributes)
        self.convs = ModuleList()
        for _ in range(num_layers):
            self.convs.append(GATv2Conv(hidden_dim, hidden_dim, edge_dim=hidden_dim, add_self_loops=False))
        
        # Final edge classification layer
        self.edge_predictor = Linear(2 * hidden_dim + hidden_dim, output_dim)

    def forward(self, x, edge_index, edge_attr):
        # Encode node and edge features
        x = self.node_encoder(x)
        x = F.relu(x)
        edge_attr = self.edge_encoder(edge_attr)
        edge_attr = F.relu(edge_attr)
        
        # Pass through GNN layers (with edge features)
        for conv in self.convs:
            x = conv(x, edge_index, edge_attr)
            x = F.relu(x)
            x = self.dropout(x)
        
        # Compute edge embeddings
        row, col = edge_index  # Source and target node indices
        edge_embedding = torch.cat([x[row], x[col], edge_attr], dim=1)  # Include edge_attr in the embedding

        # Edge classification
        logits = self.edge_predictor(edge_embedding)
        return logits.view(-1)


In [ ]:
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, f1_score
import torch.nn as nn

# ==============================
# Step 1: Model Initialization
# ==============================

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model
input_dim = train_dataset[0].x.size(-1)  # Node feature dimension
edge_dim = train_dataset[0].edge_attr.size(-1)  # Edge feature dimension
hidden_dim = 64  # Hidden layer dimension
output_dim = 1  # Binary classification (logit output)

model = EdgePredictionGNN(
    input_dim=input_dim,
    edge_dim=edge_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    num_layers=3,
    dropout=0.5
).to(device)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Adjust pos_weight for class imbalance at the **edge level**
scaled_pos_weight = 10.0  # You can tune this
loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(scaled_pos_weight).to(device))
print(f"Adjusted pos_weight: {scaled_pos_weight}")

# ==============================
# Step 2: Training Loop
# ==============================

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    all_preds, all_labels = [], []

    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()

        # Forward pass
        logits = model(batch.x, batch.edge_index, batch.edge_attr)

        # Compute loss
        loss = loss_fn(logits.view(-1), batch.y.float())
        total_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Collect predictions for metrics
        preds = (torch.sigmoid(logits).view(-1) > 0.5).long()
        all_preds.append(preds.cpu())
        all_labels.append(batch.y.cpu())

    # Concatenate predictions and labels
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    # Compute metrics
    precision = precision_score(all_labels, all_preds, zero_division=0)
    recall = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

    # Validation Phase
    model.eval()
    with torch.no_grad():
        val_loss = 0
        val_preds, val_labels = [], []

        for batch in val_loader:
            batch = batch.to(device)
            logits = model(batch.x, batch.edge_index, batch.edge_attr)
            val_loss += loss_fn(logits.view(-1), batch.y.float()).item()

            preds = (torch.sigmoid(logits).view(-1) > 0.5).long()
            val_preds.append(preds.cpu())
            val_labels.append(batch.y.cpu())

        # Concatenate validation predictions and labels
        val_preds = torch.cat(val_preds)
        val_labels = torch.cat(val_labels)

        # Compute validation metrics
        val_precision = precision_score(val_labels, val_preds, zero_division=0)
        val_recall = recall_score(val_labels, val_preds, zero_division=0)
        val_f1 = f1_score(val_labels, val_preds, zero_division=0)

        print(f"Validation Loss: {val_loss:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}")


In [ ]:
train_labels = torch.cat([graph.y for graph in train_dataset])
print(f"Train Edge Label Distribution: {torch.bincount(train_labels)}")

val_labels = torch.cat([graph.y for graph in val_dataset])
print(f"Validation Edge Label Distribution: {torch.bincount(val_labels)}")


In [ ]:
# Iterate through the first 5 graphs in the train_dataset
for i in range(5):
    graph = train_dataset[i]  # Access the i-th graph from the subset
    print(f"Graph {i}:")
    print(f" - Node Features: {graph.x}")
    print(f" - Edge Features: {graph.edge_attr}")
    print(f" - Edge Index: {graph.edge_index}")
    print(f" - Edge Labels: {graph.y}")
    print("-" * 50)


In [ ]:
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler

# Normalize node and edge features
def normalize_features(dataset):
    # Collect all node features and edge features
    all_node_features = torch.cat([data.x for data in dataset], dim=0)
    all_edge_features = torch.cat([data.edge_attr for data in dataset], dim=0)
    
    # Fit scalers
    node_scaler = StandardScaler()
    edge_scaler = StandardScaler()
    node_scaler.fit(all_node_features.numpy())
    edge_scaler.fit(all_edge_features.numpy())

    # Apply normalization to the dataset
    for data in dataset:
        data.x = torch.tensor(node_scaler.transform(data.x.numpy()), dtype=torch.float)
        data.edge_attr = torch.tensor(edge_scaler.transform(data.edge_attr.numpy()), dtype=torch.float)
    print("Node and Edge features normalized.")
    return dataset

# Normalize train, validation, and test datasets
train_dataset = normalize_features(train_dataset)
val_dataset = normalize_features(val_dataset)
test_dataset = normalize_features(test_dataset)


In [ ]:
# Calculate pos_weight based on edge label distribution
num_pos = sum([data.y.sum() for data in train_dataset])
num_neg = sum([len(data.y) - data.y.sum() for data in train_dataset])

pos_weight = num_neg / (num_pos + 1e-6)  # Add small epsilon to avoid division by zero
print(f"Positive Weight (for loss): {pos_weight:.4f}")

# Define the loss function with the updated pos_weight
loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pos_weight).to(device))


In [ ]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, NNConv

class EdgePredictionGNN(torch.nn.Module):
    def __init__(self, input_dim, edge_dim, hidden_dim, output_dim, num_layers=3, dropout=0.5):
        super(EdgePredictionGNN, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.edge_nn = Linear(edge_dim, hidden_dim)

        # Define NNConv layers for incorporating edge features
        for i in range(num_layers):
            in_dim = input_dim if i == 0 else hidden_dim
            self.convs.append(NNConv(in_dim, hidden_dim, nn=self.edge_nn, aggr='mean'))

        self.dropout = dropout
        self.output = Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index, edge_attr):
        for conv in self.convs:
            x = F.relu(conv(x, edge_index, edge_attr))
            x = F.dropout(x, p=self.dropout, training=self.training)
        return self.output(x)


In [ ]:
print(f"Length of train_dataset: {len(train_dataset)}")
print(f"Total samples requested in WeightedRandomSampler: {len(train_labels)}")


In [ ]:
train_labels = torch.cat([data.y for data in train_dataset], dim=0)
print(f"Total Edge Labels in Train Dataset: {len(train_labels)}")

# Verify the consistency of weights and indices
class_counts = torch.bincount(train_labels)
class_probs = 1.0 / (class_counts + 1e-6)
train_weights = [class_probs[label] for label in train_labels]

print(f"Number of train weights: {len(train_weights)}")

# Ensure the num_samples is valid
num_samples = len(train_weights)
train_sampler = WeightedRandomSampler(train_weights, num_samples=num_samples, replacement=True)

# Check after sampler
print(f"Sampler num_samples: {num_samples}")


In [ ]:
# Label a graph as '1' if it contains at least one positive edge, else '0'
graph_labels = torch.tensor([1 if (data.y > 0).sum() > 0 else 0 for data in train_dataset])
print(f"Graph-level Labels: {graph_labels}")

# Class counts and class weights for graph-level balancing
class_counts = torch.bincount(graph_labels)
class_weights = 1.0 / (class_counts + 1e-6)
print(f"Class Counts: {class_counts}")
print(f"Class Weights: {class_weights}")

# Assign graph weights based on their labels
graph_weights = [class_weights[label] for label in graph_labels]

# Define the sampler for graph-level oversampling
train_sampler = WeightedRandomSampler(graph_weights, num_samples=len(graph_weights), replacement=True)

# Create the DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
print("Oversampling at graph level completed.")


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class EdgePredictionGNN(nn.Module):
    def __init__(self, input_dim, edge_dim, hidden_dim, output_dim=1, num_layers=3, dropout=0.5):
        """
        GNN-based model for edge label prediction.
        Args:
            input_dim (int): Dimension of node features.
            edge_dim (int): Dimension of edge features.
            hidden_dim (int): Hidden dimension for GNN layers.
            output_dim (int): Output dimension (default: 1 for binary classification).
            num_layers (int): Number of GNN layers.
            dropout (float): Dropout rate.
        """
        super(EdgePredictionGNN, self).__init__()
        self.num_layers = num_layers
        self.dropout = dropout

        # GNN layers for processing node features
        self.convs = nn.ModuleList()
        for i in range(num_layers):
            in_dim = input_dim if i == 0 else hidden_dim
            self.convs.append(GCNConv(in_dim, hidden_dim))

        # MLP for edge classification
        self.edge_mlp = nn.Sequential(
            nn.Linear(2 * hidden_dim + edge_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x, edge_index, edge_attr):
        """
        Forward pass for the model.
        Args:
            x (torch.Tensor): Node features [num_nodes, input_dim].
            edge_index (torch.Tensor): Edge connectivity [2, num_edges].
            edge_attr (torch.Tensor): Edge features [num_edges, edge_dim].
        Returns:
            logits (torch.Tensor): Predicted logits for edges [num_edges, output_dim].
        """
        # Step 1: Generate node embeddings using GNN layers
        for conv in self.convs:
            x = conv(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, self.dropout, training=self.training)

        # Step 2: Combine source and target node embeddings for each edge
        edge_src, edge_tgt = edge_index  # Split edge_index into source and target nodes
        edge_embeddings = torch.cat([x[edge_src], x[edge_tgt], edge_attr], dim=-1)

        # Step 3: Predict edge labels
        logits = self.edge_mlp(edge_embeddings)
        return logits


In [ ]:
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, f1_score
from torch_geometric.loader import DataLoader  # Correct import for PyG DataLoader
from torch.utils.data.sampler import WeightedRandomSampler

# ==============================
# Step 1: Normalize Features
# ==============================
for graph in train_dataset + val_dataset + test_dataset:
    graph.x = (graph.x - graph.x.mean(dim=0)) / graph.x.std(dim=0).clamp(min=1e-6)
    graph.edge_attr = (graph.edge_attr - graph.edge_attr.mean(dim=0)) / graph.edge_attr.std(dim=0).clamp(min=1e-6)
print("Normalized node and edge features.")

# ==============================
# Step 2: Create DataLoader with Oversampling
# ==============================
train_graph_labels = [1 if graph.y.sum() > 0 else 0 for graph in train_dataset]  # Class 1 if graph has failed edges
class_counts = torch.bincount(torch.tensor(train_graph_labels))
class_probs = 1.0 / (class_counts + 1e-6)  # Inverse proportional to class counts
train_graph_weights = [class_probs[label] for label in train_graph_labels]
train_sampler = WeightedRandomSampler(train_graph_weights, num_samples=len(train_dataset), replacement=True)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Oversampling minority class in the training dataset.")

# ==============================
# Step 3: Model Initialization
# ==============================
input_dim = train_dataset[0].x.size(-1)  # Node feature dimension
edge_dim = train_dataset[0].edge_attr.size(-1)  # Edge feature dimension
hidden_dim = 128  # Increased hidden layer dimension for more capacity
output_dim = 1  # Binary classification (logit output)

model = EdgePredictionGNN(
    input_dim=input_dim,
    edge_dim=edge_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    num_layers=4,  # Increased number of layers
    dropout=0.5
).to('cuda' if torch.cuda.is_available() else 'cpu')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
optimizer = optim.Adam(model.parameters(),lr=1e-4, weight_decay=1e-4)  # Added weight decay

scaled_pos_weight = 5.0  # Adjusted for handling class imbalance
loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(scaled_pos_weight).to(device))
print(f"Adjusted pos_weight: {scaled_pos_weight}")

# ==============================
# Step 4: Training Loop
# ==============================
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    all_preds, all_labels = [], []

    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()

        # Forward pass
        logits = model(batch.x, batch.edge_index, batch.edge_attr)

        # Debug logits: check their distribution
        logits_mean = logits.mean().item()
        logits_min = logits.min().item()
        logits_max = logits.max().item()
        print(f"Epoch {epoch + 1}, Batch Logits - Mean: {logits_mean:.4f}, Min: {logits_min:.4f}, Max: {logits_max:.4f}")

        # Compute loss
        loss = loss_fn(logits.view(-1), batch.y.float())
        total_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Collect predictions for metrics
        preds = (torch.sigmoid(logits).view(-1) > 0.5).long()
        all_preds.append(preds.cpu())
        all_labels.append(batch.y.cpu())

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    precision = precision_score(all_labels, all_preds, zero_division=0)
    recall = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    predicted_class_counts = torch.bincount(all_preds)
    print(f"Epoch {epoch + 1}: Predicted Class Distribution: {predicted_class_counts}")

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        val_preds, val_labels = [], []
        for batch in val_loader:
            batch = batch.to(device)
            logits = model(batch.x, batch.edge_index, batch.edge_attr)

            val_logits_mean = logits.mean().item()
            val_logits_min = logits.min().item()
            val_logits_max = logits.max().item()
            print(f"Epoch {epoch + 1}, Validation Batch Logits - Mean: {val_logits_mean:.4f}, Min: {val_logits_min:.4f}, Max: {val_logits_max:.4f}")

            val_loss += loss_fn(logits.view(-1), batch.y.float()).item()
            preds = (torch.sigmoid(logits).view(-1) > 0.5).long()
            val_preds.append(preds.cpu())
            val_labels.append(batch.y.cpu())

        val_preds = torch.cat(val_preds)
        val_labels = torch.cat(val_labels)

        val_precision = precision_score(val_labels, val_preds, zero_division=0)
        val_recall = recall_score(val_labels, val_preds, zero_division=0)
        val_f1 = f1_score(val_labels, val_preds, zero_division=0)

        print(f"Validation Loss: {val_loss:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}")


In [ ]:
# Print predicted class distribution
predicted_class_counts = torch.bincount(all_preds)
print(f"Predicted Class Distribution: {predicted_class_counts}")


In [ ]:
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, f1_score

# Define the model
input_dim = train_dataset[0].x.size(-1)  # Node feature dimension
edge_dim = train_dataset[0].edge_attr.size(-1)  # Edge feature dimension
hidden_dim = 64  # Hidden layer dimension
output_dim = 1  # Binary classification (logit output)

model = EdgePredictionGNN(
    input_dim=input_dim,
    edge_dim=edge_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    num_layers=3,
    dropout=0.5
).to('cuda' if torch.cuda.is_available() else 'cpu')

# Define optimizer and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
#loss_fn = nn.BCEWithLogitsLoss()  # Use weighted loss if needed
# Adjust BCEWithLogitsLoss to handle imbalance
#loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(class_weights[1]).to(device))
# Adjust the pos_weight value to a more reasonable scale
# Scale down pos_weight further
scaled_pos_weight = 5.0  # Use a lower weight to reduce bias toward Class 1
loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(scaled_pos_weight).to(device))
print(f"Adjusted pos_weight: {scaled_pos_weight}")


# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    all_preds, all_labels = [], []

    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()

        # Forward pass
        logits = model(batch.x, batch.edge_index, batch.edge_attr)

        # Compute loss
        loss = loss_fn(logits.view(-1), batch.y.float())
        total_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Collect predictions for metrics
        preds = (torch.sigmoid(logits).view(-1) > 0.5).long()
        all_preds.append(preds.cpu())
        all_labels.append(batch.y.cpu())

    # Concatenate predictions and labels
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    # Compute metrics
    precision = precision_score(all_labels, all_preds, zero_division=0)
    recall = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        val_preds, val_labels = [], []
        for batch in val_loader:
            batch = batch.to(device)
            logits = model(batch.x, batch.edge_index, batch.edge_attr)
            val_loss += loss_fn(logits.view(-1), batch.y.float()).item()

            # Collect validation predictions and labels
            preds = (torch.sigmoid(logits).view(-1) > 0.5).long()
            val_preds.append(preds.cpu())
            val_labels.append(batch.y.cpu())

        # Concatenate validation predictions and labels
        val_preds = torch.cat(val_preds)
        val_labels = torch.cat(val_labels)

        # Compute validation metrics
        val_precision = precision_score(val_labels, val_preds, zero_division=0)
        val_recall = recall_score(val_labels, val_preds, zero_division=0)
        val_f1 = f1_score(val_labels, val_preds, zero_division=0)

        print(f"Validation Loss: {val_loss:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}")


In [ ]:
from torch_geometric.nn import SAGEConv
import torch.nn.functional as F
import torch

class EdgeClassifierWithEdgeFeatures(torch.nn.Module):
    def __init__(self, node_in_channels, edge_in_channels, hidden_channels):
        super(EdgeClassifierWithEdgeFeatures, self).__init__()
        self.conv1 = SAGEConv(node_in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.edge_fc1 = torch.nn.Linear(hidden_channels * 2 + edge_in_channels, hidden_channels)
        self.edge_fc2 = torch.nn.Linear(hidden_channels, 1)  # Single output for BCEWithLogitsLoss

    def forward(self, x, edge_index, edge_attr):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))

        src, dst = edge_index
        edge_node_features = torch.cat([x[src], x[dst]], dim=-1)
        edge_features = torch.cat([edge_node_features, edge_attr], dim=-1)

        edge_features = F.relu(self.edge_fc1(edge_features))
        return self.edge_fc2(edge_features).squeeze(-1)  # Single logit per edge


# Example initialization
model = EdgeClassifierWithEdgeFeatures(node_in_channels=3, edge_in_channels=4, hidden_channels=16)
print(model)


In [ ]:
import torch.optim as optim

# Initialize optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)

def train_model(model, data_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for data in data_loader:
            optimizer.zero_grad()
            out = model(data.x, data.edge_index, data.edge_attr)  # Forward pass
            loss = criterion(out, data.y.float())  # Convert target to float
            loss.backward()  # Backward pass
            optimizer.step()  # Optimizer step
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(data_loader):.4f}")



In [ ]:
from torch_geometric.loader import DataLoader

# Split the dataset into training and testing sets
split_ratio = 0.8
split_idx = int(split_ratio * len(processed_dataset))
train_dataset = processed_dataset[:split_idx]
test_dataset = processed_dataset[split_idx:]

# Create DataLoaders
batch_size = 32  # Adjust batch size based on your resources
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Number of training graphs: {len(train_dataset)}")
print(f"Number of testing graphs: {len(test_dataset)}")


In [ ]:
# Train the model
num_epochs = 30  # Adjust as needed
train_model(model, train_loader, criterion, optimizer, epochs=num_epochs)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import torch

def evaluate_model(model, data_loader, device):
    """
    Evaluate the model on the test/validation data loader and compute metrics.
    """
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []
    
    with torch.no_grad():  # Disable gradient computation
        for data in data_loader:
            # Move data to device (CPU/GPU)
            inputs = data.x.to(device)
            edge_index = data.edge_index.to(device)
            edge_attr = data.edge_attr.to(device)  # Edge features
            labels = data.y.to(device)  # Edge labels
            
            # Forward pass
            outputs = model(inputs, edge_index, edge_attr)
            predicted = torch.sigmoid(outputs) > 0.5  # Convert logits to binary predictions
            
            # Store predictions and true labels
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Compute metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')
    cm = confusion_matrix(all_labels, all_preds)
    
    # Print Results
    print("Evaluation Metrics:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(cm)
    
    return accuracy, precision, recall, f1, cm

# Example Usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Evaluating the model on the test dataset...")
accuracy, precision, recall, f1, cm = evaluate_model(model, test_loader, device)


In [ ]:
from sklearn.metrics import classification_report

def evaluate_bce_model(model, data_loader, threshold=0.5):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for data in data_loader:
            logits = model(data.x, data.edge_index, data.edge_attr)
            probs = torch.sigmoid(logits)  # Convert logits to probabilities
            preds = (probs >= threshold).long()  # Apply threshold
            all_preds.append(preds)
            all_targets.append(data.y)

    # Concatenate all predictions and targets
    all_preds = torch.cat(all_preds, dim=0).cpu().numpy()
    all_targets = torch.cat(all_targets, dim=0).cpu().numpy()

    # Classification report
    print(classification_report(all_targets, all_preds, digits=4))


In [ ]:
# Evaluate the model on the test set
evaluate_bce_model(model, test_loader)


In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

def evaluate_with_thresholds(model, data_loader, thresholds):
    model.eval()
    all_preds = []
    all_targets = []
    all_probs = []

    with torch.no_grad():
        for data in data_loader:
            out = model(data.x, data.edge_index, data.edge_attr)  # Forward pass
            probs = torch.softmax(out, dim=1)[:, 1]  # Probability for class 1
            all_probs.append(probs)
            all_targets.append(data.y)

    # Concatenate all predictions and targets
    all_probs = torch.cat(all_probs, dim=0).cpu().numpy()
    all_targets = torch.cat(all_targets, dim=0).cpu().numpy()

    # Evaluate at different thresholds
    results = []
    for threshold in thresholds:
        preds = (all_probs >= threshold).astype(int)
        precision, recall, f1, _ = precision_recall_fscore_support(
            all_targets, preds, average='binary', zero_division=0
        )
        results.append((threshold, precision, recall, f1))

    return results

# Define thresholds to test
thresholds = np.linspace(0.1, 0.9, 9)

# Evaluate model with varying thresholds
results = evaluate_with_thresholds(model, test_loader, thresholds)

# Print results
print("Threshold | Precision | Recall | F1-Score")
for threshold, precision, recall, f1 in results:
    print(f"  {threshold:.2f}    |  {precision:.4f}  |  {recall:.4f} |  {f1:.4f}")


In [ ]:
from sklearn.metrics import classification_report

def evaluate_model(model, data_loader):
    model.eval()
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for data in data_loader:
            out = model(data.x, data.edge_index, data.edge_attr)  # Forward pass
            preds = out.argmax(dim=1)  # Predicted class
            all_preds.append(preds)
            all_targets.append(data.y)
    
    # Concatenate all predictions and targets
    all_preds = torch.cat(all_preds, dim=0).cpu().numpy()
    all_targets = torch.cat(all_targets, dim=0).cpu().numpy()

    # Classification report
    print(classification_report(all_targets, all_preds, digits=4))

# Evaluate on the test set
evaluate_model(model, test_loader)


In [ ]:
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        """
        Focal Loss for addressing class imbalance.
        Args:
            alpha: Weighting factor for classes (tensor or list). Default is None (no weighting).
            gamma: Focusing parameter to reduce the impact of easy examples.
            reduction: Reduction method for loss ('mean', 'sum', 'none').
        """
        super(FocalLoss, self).__init__()
        self.alpha = torch.tensor(alpha, dtype=torch.float) if alpha is not None else None
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        # Compute cross-entropy loss
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)  # Probabilities of the correct class
        focal_loss = (1 - pt) ** self.gamma * ce_loss

        # Apply alpha weighting if provided
        if self.alpha is not None:
            alpha_factor = self.alpha[targets]
            focal_loss = alpha_factor * focal_loss

        # Apply reduction
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss


In [ ]:
import random

# Identify graphs containing the minority class (label 1)
def find_minority_graphs(dataset):
    minority_graph_indices = []
    for idx, data in enumerate(dataset):
        if 1 in data.y:  # Check if the graph contains the minority class
            minority_graph_indices.append(idx)
    return minority_graph_indices

minority_graphs = find_minority_graphs(train_dataset)
print(f"Number of graphs with minority class: {len(minority_graphs)}")


In [ ]:
# Custom DataLoader with oversampling
def oversampling_dataloader(dataset, minority_graphs, oversample_factor=3, batch_size=32):
    all_indices = list(range(len(dataset)))
    oversampled_indices = all_indices + minority_graphs * oversample_factor

    # Shuffle the oversampled indices
    random.shuffle(oversampled_indices)

    # Create a DataLoader with the oversampled indices
    return DataLoader(
        [dataset[i] for i in oversampled_indices],  # Oversampled dataset
        batch_size=batch_size,
        shuffle=False  # No need to shuffle, as indices are already randomized
    )

# Create the oversampling DataLoader
train_loader = oversampling_dataloader(train_dataset, minority_graphs, oversample_factor=3, batch_size=32)


In [ ]:
# Retrain the model
num_epochs = 20  # Adjust as needed
train_model(model, train_loader, criterion, optimizer, epochs=num_epochs)


In [ ]:
# Evaluate the model on the test set
evaluate_model(model, test_loader)


In [ ]:
from torch_geometric.nn import GATConv

class GATEdgeClassifier(torch.nn.Module):
    def __init__(self, node_in_channels, edge_in_channels, hidden_channels, heads=2):
        super(GATEdgeClassifier, self).__init__()
        self.gat1 = GATConv(node_in_channels, hidden_channels, heads=heads, concat=True)
        self.gat2 = GATConv(hidden_channels * heads, hidden_channels, heads=heads, concat=False)

        self.edge_fc1 = torch.nn.Linear(hidden_channels * 2 + edge_in_channels, hidden_channels)
        self.edge_fc2 = torch.nn.Linear(hidden_channels, 1)  # Single output for BCEWithLogitsLoss

    def forward(self, x, edge_index, edge_attr):
        x = F.elu(self.gat1(x, edge_index))
        x = F.elu(self.gat2(x, edge_index))

        src, dst = edge_index
        edge_node_features = torch.cat([x[src], x[dst]], dim=-1)
        edge_features = torch.cat([edge_node_features, edge_attr], dim=-1)

        edge_features = F.relu(self.edge_fc1(edge_features))
        return self.edge_fc2(edge_features).squeeze(-1)  # Output logits as 1D tensor


In [ ]:
# Initialize the GAT model
#model = GATEdgeClassifier(node_in_channels=3, edge_in_channels=4, hidden_channels=16, heads=2)
model = GATEdgeClassifier(node_in_channels=3, edge_in_channels=4, hidden_channels=32, heads=4)

# Reinitialize the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.05)


In [ ]:
# Train the GAT model
num_epochs = 30  # Adjust as needed
train_model(model, train_loader, criterion, optimizer, epochs=num_epochs)


In [ ]:
# Evaluate the tuned GAT model
evaluate_bce_model(model, test_loader)


In [ ]:
from torch_geometric.nn import GATConv
import torch
import torch.nn.functional as F
from torch.nn import Linear

class EdgeGAT(torch.nn.Module):
    def __init__(self, node_in_channels, edge_in_channels, hidden_channels, heads=4):
        super(EdgeGAT, self).__init__()
        self.gat1 = GATConv(node_in_channels, hidden_channels, heads=heads, concat=True)
        self.gat2 = GATConv(hidden_channels * heads, hidden_channels, heads=heads, concat=False)

        # Edge-specific fully connected layers
        self.edge_fc1 = Linear(hidden_channels * 2 + edge_in_channels, hidden_channels)
        self.edge_fc2 = Linear(hidden_channels, 1)  # Single output for BCEWithLogitsLoss

    def forward(self, x, edge_index, edge_attr):
        # Node embeddings via GAT layers
        x = F.elu(self.gat1(x, edge_index))
        x = F.elu(self.gat2(x, edge_index))

        # Prepare edge embeddings
        src, dst = edge_index
        edge_node_features = torch.cat([x[src], x[dst]], dim=-1)
        edge_features = torch.cat([edge_node_features, edge_attr], dim=-1)

        # Pass through edge-specific layers
        edge_features = F.relu(self.edge_fc1(edge_features))
        return self.edge_fc2(edge_features).squeeze(-1)  # Single logit per edge


In [ ]:
# Initialize the Edge-GAT model
model = EdgeGAT(node_in_channels=3, edge_in_channels=4, hidden_channels=32, heads=4)

# Reinitialize the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.005)


In [ ]:
# Train the Edge-GAT model
num_epochs = 15  # Adjust as needed
train_model(model, train_loader, criterion, optimizer, epochs=num_epochs)


In [ ]:
# Evaluate the Edge-GAT model
evaluate_bce_model(model, test_loader)


In [ ]:
def create_edge_oversampling_loader(dataset, batch_size=32, oversample_factor=3):
    """
    Create a DataLoader with dynamic edge-level oversampling.
    Args:
        dataset: List of Data objects.
        batch_size: Number of graphs per batch.
        oversample_factor: Multiplier for minority-class edges.
    Returns:
        DataLoader with oversampled edges.
    """
    oversampled_data = []

    for data in dataset:
        # Separate edges by class
        minority_edges = (data.y == 1).nonzero(as_tuple=True)[0]
        majority_edges = (data.y == 0).nonzero(as_tuple=True)[0]

        # Oversample minority edges
        if len(minority_edges) > 0:  # Avoid empty minority class
            oversampled_minority_edges = minority_edges.repeat(oversample_factor)
            oversampled_edges = torch.cat([majority_edges, oversampled_minority_edges])

            # Shuffle edges
            shuffled_indices = torch.randperm(len(oversampled_edges))
            oversampled_edges = oversampled_edges[shuffled_indices]

            # Update edge attributes and labels
            data.edge_index = data.edge_index[:, oversampled_edges]
            data.edge_attr = data.edge_attr[oversampled_edges]
            data.y = data.y[oversampled_edges]

        oversampled_data.append(data)

    return DataLoader(oversampled_data, batch_size=batch_size, shuffle=True)

# Create a DataLoader with edge-level oversampling
train_loader = create_edge_oversampling_loader(train_dataset, batch_size=32, oversample_factor=3)


In [ ]:
# Retrain the Edge-GAT model with oversampled edges
train_model(model, train_loader, criterion, optimizer, epochs=15)


In [ ]:
# Evaluate the Edge-GAT model with oversampled edges
evaluate_bce_model(model, test_loader)


In [ ]:
import torch.nn.functional as F
import torch

class ClassBalancedFocalLoss(torch.nn.Module):
    def __init__(self, beta=0.9999, gamma=2.0):
        """
        Class-Balanced Focal Loss for handling severe class imbalance.
        Args:
            beta: Hyperparameter to control class weighting (near 1.0 for large datasets).
            gamma: Focusing parameter to adjust the contribution of easy and hard examples.
        """
        super(ClassBalancedFocalLoss, self).__init__()
        self.beta = beta
        self.gamma = gamma

    def forward(self, inputs, targets):
        """
        Args:
            inputs: Logits from the model.
            targets: Ground truth labels (0 or 1).
        Returns:
            Loss value computed for the batch.
        """
        # Compute effective number of samples
        num_samples = targets.size(0)
        class_counts = torch.bincount(targets.long(), minlength=2)
        effective_num = (1.0 - self.beta) / (1.0 - self.beta**class_counts.float())

        # Class weights
        class_weights = effective_num / effective_num.sum()

        # Logits to probabilities
        probs = torch.sigmoid(inputs)
        probs = probs * targets + (1 - probs) * (1 - targets)  # Adjust for binary case
        focal_weights = (1 - probs) ** self.gamma

        # Apply class weights
        class_weight = class_weights[targets.long()]
        loss = -class_weight * focal_weights * torch.log(probs + 1e-8)

        return loss.mean()


In [ ]:
# Initialize the class-balanced focal loss
criterion = ClassBalancedFocalLoss(beta=0.9999, gamma=2.0)
print("Using Class-Balanced Focal Loss with beta=0.9999 and gamma=2.0")


In [ ]:
# Retrain the model with Class-Balanced Focal Loss
train_model(model, train_loader, criterion, optimizer, epochs=15)


In [ ]:
# Evaluate the Edge-GAT model with Class-Balanced Focal Loss
evaluate_bce_model(model, test_loader)


In [ ]:
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    balanced_accuracy_score,
    f1_score,
)

def evaluate_model_with_metrics(model, data_loader, threshold=0.5):
    model.eval()
    all_preds = []
    all_targets = []
    all_logits = []

    with torch.no_grad():
        for data in data_loader:
            logits = model(data.x, data.edge_index, data.edge_attr)
            probs = torch.sigmoid(logits)  # Convert logits to probabilities
            preds = (probs >= threshold).long()  # Apply threshold
            all_preds.append(preds)
            all_logits.append(probs)
            all_targets.append(data.y)

    # Concatenate all predictions and targets
    all_preds = torch.cat(all_preds, dim=0).cpu().numpy()
    all_targets = torch.cat(all_targets, dim=0).cpu().numpy()
    all_logits = torch.cat(all_logits, dim=0).cpu().numpy()

    # Classification report
    print("Classification Report:\n", classification_report(all_targets, all_preds, digits=4))

    # Additional Metrics
    auc = roc_auc_score(all_targets, all_logits)
    balanced_acc = balanced_accuracy_score(all_targets, all_preds)
    f1_class_1 = f1_score(all_targets, all_preds, pos_label=1)

    print(f"AUC-ROC: {auc:.4f}")
    print(f"Balanced Accuracy: {balanced_acc:.4f}")
    print(f"F1-Score for Class 1: {f1_class_1:.4f}")


In [ ]:
# Evaluate the model with new metrics
evaluate_model_with_metrics(model, test_loader)


In [ ]:
def apply_smote_to_edges_safe(dataset):
    """
    Apply SMOTE to oversample the minority class at the edge level
    while maintaining node index consistency.
    Args:
        dataset: List of torch_geometric.data.Data objects.
    Returns:
        A new dataset with SMOTE-applied edge-level augmentation.
    """
    augmented_dataset = []

    for data in dataset:
        edge_features = data.edge_attr.cpu().numpy()
        edge_labels = data.y.cpu().numpy()

        # Apply SMOTE only if the minority class is present
        if len(set(edge_labels)) > 1:
            smote = SMOTE(k_neighbors=5)
            edge_features_smote, edge_labels_smote = smote.fit_resample(edge_features, edge_labels)

            # Update edge attributes and labels
            data.edge_attr = torch.tensor(edge_features_smote, dtype=torch.float)
            data.y = torch.tensor(edge_labels_smote, dtype=torch.long)

            # Limit edge_index to the original number of nodes
            num_edges = data.edge_attr.size(0)
            edge_index = torch.randint(0, data.x.size(0), (2, num_edges))  # Randomly assign edges
            data.edge_index = edge_index

        augmented_dataset.append(data)

    return augmented_dataset

# Apply the updated SMOTE function to the training dataset
train_dataset_smote = apply_smote_to_edges_safe(train_dataset)

# Create a DataLoader for the SMOTE-augmented dataset
train_loader_smote = DataLoader(train_dataset_smote, batch_size=32, shuffle=True)


In [ ]:
# Retrain the Edge-GAT model with corrected SMOTE-augmented dataset
train_model(model, train_loader_smote, criterion, optimizer, epochs=15)


In [ ]:
# Evaluate the Edge-GAT model
evaluate_model_with_metrics(model, test_loader)
